In [15]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from datetime import datetime
import re

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [16]:
tr = pd.read_csv('input/train.csv', low_memory=False)
tr['Id'] = np.nan
del tr['Customers']
#tr[:5]

In [17]:
te = pd.read_csv('input/test.csv', low_memory=False)
te['Sales'] = np.nan
#te[:5]

In [18]:
s = pd.read_csv('input/store.csv')
s.index = s.Store
s.index.name = 'StoreId'
#s[:5]

In [19]:
d = pd.concat([tr, te])

In [20]:
def parse_year(d):
    return int(re.match(r'(\d+)-(\d+)-(\d+)', d).groups()[0])
def parse_month(d):
    return int(re.match(r'(\d+)-(\d+)-(\d+)', d).groups()[1])
def parse_day(d):
    return int(re.match(r'(\d+)-(\d+)-(\d+)', d).groups()[2])
d['DateYear'] = d.Date.apply(lambda d: parse_year(d))
d['DateMonth'] = d.Date.apply(lambda d: parse_month(d))
d['DateDay'] = d.Date.apply(lambda d: parse_day(d))
#d[:5]

In [21]:
def agg_store(g):
    ret = DataFrame({
            'StoreSales_5': [g.Sales.quantile(0.05)],
            'StoreSales_25': [g.Sales.quantile(0.25)],
            'StoreSales_50': [g.Sales.quantile(0.50)],
            'StoreSales_75': [g.Sales.quantile(0.75)],
            'StoreSales_95': [g.Sales.quantile(0.95)]
        })
    return ret
g = d.groupby(['Store'])
g = g.apply(agg_store)
g['Store'] = g.index.get_level_values(0)
g.index = g.index.get_level_values(0)
g.index.name = 'StoreId'
t = pd.merge(s, g, how='left', on='Store')
#t[:5]

In [22]:
def agg_storeday(h):
    ret = DataFrame({
            'StoreDaySales_5': [h.Sales.quantile(0.05)],
            'StoreDaySales_25': [h.Sales.quantile(0.25)],
            'StoreDaySales_50': [h.Sales.quantile(0.50)],
            'StoreDaySales_75': [h.Sales.quantile(0.75)],
            'StoreDaySales_95': [h.Sales.quantile(0.95)]
        })
    return ret
h = d.groupby(['Store', 'DayOfWeek'])
h = h.apply(agg_storeday)
h['Store'] = h.index.get_level_values(0)
h['DayOfWeek'] = h.index.get_level_values(1)
e = pd.merge(d, h, how='left', on=['Store', 'DayOfWeek'])
e[:5]

,Date,DayOfWeek,Id,Open,Promo,Sales,SchoolHoliday,StateHoliday,Store,DateYear,DateMonth,DateDay,StoreDaySales_25,StoreDaySales_5,StoreDaySales_50,StoreDaySales_75,StoreDaySales_95
0,2015-07-31,5,NaN,1,1,5263,1,0,1,2015,7,31,4001.5,3239.3,4609,5228.5,6140.5
1,2015-07-31,5,NaN,1,1,6064,1,0,2,2015,7,31,3954.5,3050.9,4608,4982.0,6003.9
2,2015-07-31,5,NaN,1,1,8314,1,0,3,2015,7,31,5892.0,3042.2,6933,8065.5,9624.1
3,2015-07-31,5,NaN,1,1,13995,1,0,4,2015,7,31,7970.5,6524.2,9299,10280.0,13089.5
4,2015-07-31,5,NaN,1,1,4822,1,0,5,2015,7,31,4178.5,2461.2,4764,5204.0,6262.5


In [23]:
f = pd.merge(e, t, how='left', on='Store')
#f[:5].T

In [24]:
def month_in_interval(m, i):
    if i is not None and i != nan and (type(i) == str and (m in i)):
        return 1
    return 0
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for m in months:
    f['PromoInterval_' + m] = f.PromoInterval.apply(lambda i: month_in_interval(m, i))
#f[:5].T

In [25]:
#f.info()

In [26]:
g = pd.get_dummies(f, columns=['DayOfWeek', 'StoreType', 'Assortment', 'StateHoliday'])
#g.info()

In [27]:
del g['PromoInterval']

In [28]:
g.to_csv('tmp/engineered_features_2.csv')